<a href="https://colab.research.google.com/github/nkcong206/AskUSTH/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain-chroma
!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchainhub
!pip install sentence_transformers
!pip install langchain-huggingface

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.65.4 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.4 which is incompatible.


In [ ]:
!pip install qdrant_client
!pip install -qU langchain-google-genai

  Using cached protobuf-5.27.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.27.3-cp38-abi3-manylinux2014_x86_64.whl (309 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.4
    Uninstalling protobuf-4.25.4:
      Successfully uninstalled protobuf-4.25.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.26.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.3 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.65.4 re

In [ ]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://da9fadd2-dc5a-4481-ac79-4e2677a2354b.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="t6C4zWq3Can8Grxjzs-GTRyAjd0Tm_arS-IKltIJQQREwMC5pCnlgw",
)

print(qdrant_client.get_collections())

collections=[]


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain import hub
from langchain.document_loaders import TextLoader
import os
from langchain_ollama.llms import OllamaLLM
from langchain_community.vectorstores import Qdrant

In [ ]:
folder_path = "/home/cong/Projects/LLMs/RAG"

# Initialize an empty list to store documents
documents = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        loader = TextLoader(file_path, encoding="utf8")
        documents.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

In [ ]:
print(docs)

[Document(metadata={'source': '/home/cong/Projects/LLMs/RAG/tuyensinh.txt'}, page_content='\ufeffTôi tên là Nguyễn Khắc Công, sinh viên ngành khoa học dữ liệu trường đại học khoa học và Công nghệ Hà nội (USTH)\n\nKhoa Nước – Môi trường – Hải dương học\nĐịa chỉ: Phòng 306, tầng 3, tòa nhà A21, Trường Đại học Khoa học và Công nghệ Hà Nội\nTel: (+84-24) 37 91 72 06\nEmail: weo_dept@usth.edu.vn'), Document(metadata={'source': '/home/cong/Projects/LLMs/RAG/tuyensinh.txt'}, page_content='Tuyển sinh\nKhoa Nước – Môi trường – Hải dương học được thành lập năm 2011 dưới sự liên kết giữa chính phủ Việt Nam với các trường đại học và tổ chức nghiên cứu của Pháp.\nHiện tại, Khoa tuyển sinh các chương trình sau:\nĐại học: Khoa học Môi trường Ứng dụng (Mã ngành: 7440301)\nThạc sĩ: Khoa học Môi trường Ứng dụng\nTiến sĩ: ngành Nước – Môi trường – Hải dương học'), Document(metadata={'source': '/home/cong/Projects/LLMs/RAG/tuyensinh.txt'}, page_content='Để biết thêm thông tin chi tiết về chính sách tuyển 

In [ ]:
model_name="vinai/phobert-base-v2"


model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/home/cong/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name vinai/phobert-base-v2. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/cong/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more d

In [ ]:
db = Qdrant(client=qdrant_client, embeddings=embeddings, collection_name="vector_db")

/home/cong/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


In [ ]:
db1 = Qdrant.from_documents(docs, embeddings,
                            path="/home/cong/Projects/LLMs/RAG/2222",
                            collection_name="test")

In [ ]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
model = OllamaLLM(model="ontocord/vistral:latest")
os.environ["LANGCHAIN_API_KEY"] = "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"

In [ ]:
from langchain.prompts import PromptTemplate
def creat_prompt(template):
    prompt = PromptTemplate(template = template, input_variables=["context", "question"])
    return prompt
template = """<|im_start|>system\nBản là một trợ lí AI hỗ trợ tuyển sinh và sinh viên. Hãy trả lời câu hỏi chính xác, tập trung vào thông tin liên quan đến câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời.
\n

    Dưới đây là thông tin liên quan mà bạn có thể sử dụng:\n{context}<|im_end|>\n<|im_start|>hãy trả lời: \n{question}<|im_end|>\n<|im_start|>assistant"""

prompt = creat_prompt(template)

In [ ]:
llm_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type= "stuff",
    retriever = db1.as_retriever(search_kwargs = {"k":2}),
    return_source_documents = True,
    chain_type_kwargs= {'prompt': prompt}
)
# max_tokens_limit=4096

Đặt câu hỏi truy vấn

In [ ]:
question = "CHƯƠNG TRÌNH GIẢNG DẠY của ngànhKỹ thuật vận hành là gì?"
result = llm_chain.invoke({"query": question ,})
result

{'query': 'CHƯƠNG TRÌNH GIẢNG DẠY của ngànhKỹ thuật vận hành là gì?',
 'result': 'Tôi không biết. <|im_end|>',
 'source_documents': [Document(metadata={'source': '/home/cong/Projects/LLMs/RAG/tuyensinh.txt', '_id': '5231777893c145bdb6af4d374149361b', '_collection_name': 'test'}, page_content='Ngành Hóa học\nChương trình đào tạo trình độ đại học ngành hóa học tại USTH được xây dựng nhằm đào tạo nguồn nhân lực chất lượng cao cho ngành hóa học tại Việt Nam. Chương trình được thiết kế đảm bảo tính chọn lọc, hiện đại trong các nội dung khoa học và phương pháp sư phạm.\nMục lục\n[Ẩn]\n1. GIỚI THIỆU CHƯƠNG TRÌNH\n2. CHƯƠNG TRÌNH GIẢNG DẠY\n3. THỰC TẬP\n4. ĐẦU RA CỦA CHƯƠNG TRÌNH\n5. CƠ HỘI NGHỀ NGHIỆP\n6. THÔNG TIN LIÊN HỆ\n1. GIỚI THIỆU CHƯƠNG TRÌNH'),
  Document(metadata={'source': '/home/cong/Projects/LLMs/RAG/tuyensinh.txt', '_id': '59577769884741e88804b97de30e37d8', '_collection_name': 'test'}, page_content='Ngành Toán ứng dụng\nChương trình cử nhân ngành Toán ứng dụng của USTH được thiế